In [11]:
%load_ext autoreload
%autoreload 2

import teetool as tt
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("data/eddm_eddh.csv")
cluster_data_list = []

cluster_data = []

for trajectory_id in df.flightid.unique():
    dft = df[df["flightid"] == trajectory_id]
    Yn = np.array([
        dft["groundspeed"].values, 
        dft["altitude"].values
    ]).T
    
    (npoints, _) = Yn.shape
    xn = dft["dist_to_thr"].values
    
    cluster_data.append((xn, Yn))

print("---------- + \n     {0}".format(len(cluster_data)))

In [16]:
world = tt.World(name="toy", ndim=2)

# this normalizes x (dist to thr.) in interval [0,1]
world.addCluster(cluster_data, "data")

In [14]:
visual = tt.visual_2d.Visual_2d(world)
visual.plotTrajectories(ntraj=50)
visual.show()

In [15]:
# visual
visual = tt.visual_2d.Visual_2d(world)
visual.plotTimeSeries(icluster=0, idim=0, ntraj=50)
visual._ax.set_xlabel(r"$\tau$ [-]")
visual._ax.set_ylabel("gs [m/s]")
visual.show()

visual = tt.visual_2d.Visual_2d(world)
visual.plotTimeSeries(icluster=0, idim=1, ntraj=50)
visual._ax.set_xlabel(r"$\tau$ [-]")
visual._ax.set_ylabel("altitude [m]")
visual.show()

In [14]:
def build_model(model_type, basis_type, nbasis, ngaus):
    # build the model
    settings = {"model_type": model_type,
                "basis_type": basis_type,
                "nbasis": nbasis,
                "ngaus":ngaus}    
    world.buildModel(settings)

def world_observe(nobs):
    x = np.array([1.0, 0.95, 0.9, 0.8, 0.7, 0.6, 0.5, 0.2])
    y = np.array([
        # [.6, .6], # 1.00
        # [.6, .6], # 0.95
       [.6 * (180.5 - 55.3) + 55.3, .6 * (2445.0 - 478.0) + 478.0], # 1.00
       [.6 * (180.5 - 55.3) + 55.3, .6 * (2445.0 - 478.0) + 478.0], # 0.95
    ])
    world.addObservation(icluster=0, observation = (x[:nobs], y[:nobs].reshape(-1,1, order='F')) )
    
def plot_world(idim=None):
    visual = tt.visual_2d.Visual_2d(world)
    visual.plotSamples(ntraj=50, idim=idim)
    if idim is None:
        visual.plotTube(sdwidth=2, resolution=[500,500,500])
    visual.show()

In [19]:
build_model("ML", "bernstein", 10, 100)

#world_observe(1)

plot_world(0), plot_world(1), plot_world();
print(world.getOutline())

In [ ]:
def analyze():
    #world.getTubeStats()
    mds = world.getMarginalDistribution(x = 0.0)
    m, S = mds[0]
    
    m = np.asarray(m).reshape(-1)
    S = S
    return m[0], S[0]

x, y = [], []

for i in range(0, 8):
    world_observe(i)
    m, s = analyze()
    x.append(m)
    y.append(s)
    
#plt.plot(range(0,8), x)
plt.plot(range(0,8), y)
plt.show()